In [1]:
import os
import sys
import cv2

import random

import numpy as np

from tqdm import tqdm
import pickle

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense

from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [ ]:
train_images = os.listdir('data/train')
test_images = os.listdir('data/test')

print(len(train_images))
print(len(test_images))

In [ ]:
train_images[0]

In [ ]:
test_images_data = []

for image in tqdm(test_images):
    image_data = cv2.imread('data/test/' + image)

    #convert color from BGR to RGB
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)    
    image_data = cv2.resize(image_data, (64, 64))
    
    test_images_data.append(image_data)

In [ ]:
test_images_data = np.array(test_images_data)

In [ ]:
train_images_data = []
train_images_labels = []

random.shuffle(train_images)

for image in tqdm(train_images):
    image_data = cv2.imread('data/train/' + image)
    
    #convert color from BGR to RGB
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    image_data = cv2.resize(image_data, (64, 64))

    train_images_data.append(image_data)
    train_images_data.append(image_data[:, ::-1]) #flipped image
    
    if image.startswith('cat'):
        train_images_labels.append(0)
        train_images_labels.append(0)
    else:
        train_images_labels.append(1)
        train_images_labels.append(1)

In [ ]:
train_images_data = np.array(train_images_data)
train_images_labels = np.array(train_images_labels)

train_images_data = train_images_data.reshape([-1, 64, 64, 3])

In [ ]:
pickle.dump(train_images_data, open('model_data/colored_train_images_data.pck', 'wb'))

In [ ]:
pickle.dump(test_images_data, open('model_data/colored_test_images_data.pck', 'wb'))

In [ ]:
pickle.dump(train_images_labels, open('model_data/colored_train_images_labels.pck', 'wb'))

In [2]:
file = open('model_data/colored_test_images_data.pck','rb')
test_images_data = pickle.load(file)

In [3]:
file = open('model_data/colored_train_images_data.pck','rb')
train_images_data = pickle.load(file)

In [4]:
file = open('model_data/colored_train_images_labels.pck','rb')
train_images_labels = pickle.load(file)

In [10]:
model_name = 'colored_model_conv_5_by_5_convs(64_32_32)_last_dense_50_dropout_10%' #

In [11]:
model = Sequential() 
model.add(Conv2D(64, (5, 5), input_shape=(64, 64, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.1)) 
  
model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
#model.add(Dropout(0.1)) 
  
model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
#model.add(Dropout(0.1)) 
  
model.add(Flatten()) 
model.add(Dense(50)) #Increase to see how accuracy performs 
#consider also with 0.1 dropout it went from training with 25 to training with 45

model.add(Activation('relu')) 
model.add(Dropout(0.1)) #compare 0.5 with to 0.1 then 0.2 and 0.3
model.add(Dense(1)) 
model.add(Activation('sigmoid')) 

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']) 

tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))
early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint(filepath='models/' + model_name + '.h5', monitor='val_loss', 
                             save_best_only=True)

model.fit(train_images_data, train_images_labels, epochs=100, validation_split=0.1, 
          callbacks=[tensorboard, early_stop, checkpoint]) #move batch size to 32, 64, 128
#and finally 256 see how accuracy performs

#remember to try out with dropout per layer

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
45000/45000 [==============================] - 781s 17ms/step - loss: 0.7124 - accuracy: 0.5362 - val_loss: 0.6840 - val_accuracy: 0.5298
Epoch 2/100
45000/45000 [==============================] - 695s 15ms/step - loss: 0.6812 - accuracy: 0.5526 - val_loss: 0.6788 - val_accuracy: 0.5486
Epoch 3/100
45000/45000 [==============================] - 1038s 23ms/step - loss: 0.6772 - accuracy: 0.5636 - val_loss: 0.6638 - val_accuracy: 0.6052
Epoch 4/100
45000/45000 [==============================] - 1104s 25ms/step - loss: 0.6757 - accuracy: 0.5630 - val_loss: 0.6865 - val_accuracy: 0.5638
Epoch 5/100
45000/45000 [==============================] - 1101s 24ms/step - loss: 0.6505 - accuracy: 0.6203 - val_loss: 0.6302 - val_accuracy: 0.6446
Epoch 6/100
45000/45000 [==============================] - 1098s 24ms/step - loss: 0.6018 - accuracy: 0.6785 - val_loss: 0.6066 - val_accuracy: 0.6884
Epoch 7/100
45000/45000 [======================

In [9]:
model.save('models/' + model_name + '.h5')